In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

# 0 - Read prepped cohorts data

- Read in prepared data and output

In [ ]:
results <- fread(paste0(SHARE_DIR, "2b_go.csv")) 
fisher_base <- fread(paste0(SHARE_DIR, "fisher_base.csv"))

- Collect top independent features

In [ ]:
results <- 
results %>% mu( p_bh = p.adjust(fisher_pval, method = "BH"), p_by = p.adjust(fisher_pval, method = "BY")) %>%
 gb(cohortGo) %>% mu(max_patients = max(total_patients))

In [ ]:
dim(results)

In [ ]:
tots <- 
results %>% 
 gb(cohortGo, max_patients, type) %>% 
 su( p_sig = sum(fisher_pval < .05), 
     p_bh_sig = sum(p_bh < .05),
     p_by_sig = sum(p_by < .05), 
     tests = n(), 
     type1 = p_sig/tests, 
     lt_05 = type1 < .05) %>% 
 ug() %>% 
 ar(p_bh_sig) 

In [ ]:
#tots

- Highlighters

In [ ]:
highlighters_univariate <-
c(
 "Pan-Cancer ## Pazopanib" = "rna_geneset_KEGG_PHENYLALANINE_METABOLISM_lt25", 
 "Prostate ## Abiraterone" = "rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt25", 
 "Breast ER+/HER- ## Capecitabine" = "cn_simple_chr21_p_gt75",
 "Pan-Cancer ## Nivolumab" = "rna_geneset_gene_set_t_cell_gep_6_lt25", 
 "Lung NSCLC ## Anti-PD-1" = "neo_ct_lt75", 
 "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50", 
 "Pan-Cancer ## Aromatase inhibitor" = "rna_geneset_HALLMARK_ANDROGEN_RESPONSE_lt50", 
 "Pan-Cancer ## Anti-EGFR" = "drivers_pathway_perplexity_total_gt75",
 "Colorectum ## Chemotherapy" = "hotspot_KRAS_position_25398284", 
 "Skin Melanoma ## Immunotherapy" = "drivers_pathway_IMMUNE_EVASION_gt0", 
 "Breast Triple Negative ## Chemotherapy" = "rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50",
 "Lung NSCLC ## Chemotherapy" = "drivers_pathway_NRF2_gt0"    
)

In [ ]:
results %>% 
 fi(e_nr > 10, e_r < 1, type != "univariate" ) %>% 
 gb(cohortGo, group) %>% 
 mu(rk = row_number(fisher_pval)) %>% 
 ug() %>% 
 fi(rk <= 4) %>% 
 ar(group, fisher_pval) %>% 
 se(group, cohortGo, feature, e_nr, e_r, ne_nr, ne_r, fisher_pval, p_bh, p_by) %>% 
 fi(group == "type") #%>% 
# fi(grepl("driver", feature))

- Interation Event to Highlight

In [ ]:
highlighters_combo <- 
c( "Pan-Cancer ## Alkaloid ## Platinum" = "cn_simple_chr11_q_gt75_or_purity_msIndelsPerMb_gt75", 
   "Prostate ## Anti-AR" = "chord_BRCA1_gt0_or_rna_geneset_KEGG_COMPLEMENT_AND_COAGULATION_CASCADES_gt25", 
   "Colon ## Chemotherapy" = "hotspot_KRAS_position_25398284_and_purity_tmbPerMb_gt50", 
   "Pan-Cancer ## Multikinase inhibitor" = "cn_simple_chr17_p_lt75_and_rna_geneset_KEGG_PROPANOATE_METABOLISM_lt25", 
   "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_OXIDATIVE_PHOSPHORYLATION_lt75_and_rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50",
   "Lung NSCLC ## Anti-PD-1" = "cn_simple_chr3_p_lt75_and_neo_ct_lt75", 
   "Breast Triple Negative ## Chemotherapy" = "cn_simple_chr22_q_gt25_and_rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50"
 )

In [ ]:
highlighters_individual <- 
c("Colon ## Chemotherapy" = "hotspot_KRAS_position_25398284", 
  "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50", 
  "Pan-Cancer ## Multikinase inhibitor" = "rna_geneset_KEGG_ERBB_SIGNALING_PATHWAY_gt75", 
  "Pan-Cancer ## Targeted therapy" = "driver_KRAS", 
  "Skin Melanoma ## Immunotherapy" = "drivers_pathway_IMMUNE_EVASION_gt0",
  "Prostate ## Taxane" = "cn_simple_chr18_q_lt25"
 )

In [ ]:
now <- c()
for( i in names(highlighters_univariate)){
    now <- c(now, paste0(i, "--", highlighters_univariate[i]))
}

In [ ]:
now2 <- c()
for( i in names(highlighters_individual)){
    now2 <- c(now2, paste0(i, "--", highlighters_individual[i]))
}

In [ ]:
map <- c("e_nr" = "Event + No DCB", "e_r" = "Event + DCB", "ne_nr" = "No Event + No DCB", "ne_r" = "No Event + DCB")

In [ ]:
tmp <- 
results %>% 
 se(cohortGo, feature, e_nr, ne_nr, e_r, ne_r, p_bh, group) %>% 
 ga(event, ct, -cohortGo, -feature, -p_bh, -group) %>% 
 mu(event2 = factor(map[event], levels = unname(map)),  event = !m("No Event", event2), dcb = !m("No DCB", event2)) %>% 
 rw() %>% mu(then = paste0(cohortGo, "--", feature)) %>% ug() %>% 
 #fi( (cohortGo %in% names(highlighters)) & (feature %in% unlist(highlighters))) %>%
 fi( then %in% now) %>%
 #fi( then %in% now2) %>%
 gb(cohortGo, feature) %>% 
 mu(nr = max(ct*(as.numeric(event2 == "Event + No DCB")))) %>% 
 ug()

In [ ]:
#head(tmp)

In [ ]:
alphas <- c("FALSE" = .4, "TRUE" = 1)
colors <- c("FALSE" = "grey", "TRUE" = "black")
response <- c("FALSE" = "#e52f28", "TRUE" = "#7AABD3")

In [ ]:
options(repr.plot.width =13, repr.plot.height = 4)

In [ ]:
head(tmp)

In [ ]:
plotter <- function( df, label = "Treatment Examples\n", theme = go_theme, ylab_pt = 30 ){
 
 ggplot(data = df, 
        aes(x = event2, y = ct, alpha = event, fill = dcb, color = event)) + 
  facet_wrap(~fct_reorder(cohortGo, p_bh, .fun = mean) + 
              str_wrap(gsub("_", " ", gsub("rna_geneset_", "",feature)), width = 45) , ncol = 4) +    
  geom_bar(stat = "identity") + 
  scale_fill_manual( values = response) +  
  scale_alpha_manual( values = alphas) + 
  scale_color_manual( values = colors) + 
  geom_text(aes(label = ct), vjust = -0.5, color = "black", size = 3) + 
  scale_y_continuous(
    name = "Number of Patients\n",
    sec.axis = sec_axis(~ ., name = label), 
    expand = expansion(mult = c(0.1, 0.1))
  ) + 
  geom_text(aes(x = 1.3, y = ylab_pt, label = paste0("Adjusted p-value:\n", formatC(p_bh, format = "e", digits = 1))), vjust = -0.5, color = "black", size = 3) + 
  theme
}

In [ ]:
top_mid_theme <-
go_theme + 
 theme(strip.background = element_rect(fill = "white", color = "black"), 
       legend.position = "none", 
       axis.title.x = element_blank(),
       axis.text.x = element_blank(),
       axis.ticks.x = element_blank())

In [ ]:
bottom_theme <-
go_theme + 
 theme(strip.background = element_rect(fill = "white", color = "black"), 
       legend.position = "none")

In [ ]:
top <- plotter(tmp %>% fi(group == "treatment"), theme = top_mid_theme, label = "Treatment Examples\n")
mid <- plotter(tmp %>% fi(group == "mechanism"), theme = top_mid_theme, label = "Mechanism Examples\n")
bottom <- plotter(tmp %>% fi(group == "type"), theme = bottom_theme, label = "Type Examples\n", ylab_pt = 75)

In [ ]:
#ggsave( paste0(FIG_DIR, "Combination_events_with_no_durable_benefit.png"), plot = top, width = 12, height = 7)

In [ ]:
library(patchwork)

In [ ]:
cohort <- (top / mid / bottom ) + plot_layout(heights = c(5, 7, 8)) 

In [ ]:
oo = (cohort | pan_cancer ) + plot_layout(widths = c(1,1)) #+ plot_annotation(title = "Overall Title", subtitle = "Subtitle here")

# Generally Bad Prognostic Features 

In [ ]:
type_map <- 
c('Bladder Urothelial ## Anti-PD-1' = 'Immunotherapy',
  'Bladder Urothelial ## Immunotherapy' = 'Immunotherapy',
  'Breast ER+/HER- ## Aromatase inhibitor' = "Hormonal",
  'Breast ER+/HER- ## Chemotherapy' = "Chemotherapy",
  'Breast ER+/HER- ## Hormonal therapy' = "Hormonal",
  'Breast ER+/HER- ## Hormonal therapy ## Targeted therapy' = "Targeted",
  'Breast ER+/HER- ## Pyrimidine (ant)agonist' = "Chemotherapy",
  'Breast Triple Negative ## Chemotherapy'  = "Chemotherapy",
  'Colon ## Chemotherapy' = "Chemotherapy",
  'Colon ## Chemotherapy ## Targeted therapy' = "Chemotherapy",
  'Gastroesophageal ## Chemotherapy' = "Chemotherapy",
  'Kidney ## Multikinase inhibitor' = "Targeted",
  'Kidney ## Targeted therapy' = "Targeted",
  'Lung NSCLC ## Anti-PD-1' = 'Immunotherapy',
  'Lung NSCLC ## Chemotherapy' = "Chemotherapy",
  'Lung NSCLC ## Immunotherapy' = 'Immunotherapy',
  'Ovary ## Chemotherapy' = "Chemotherapy",
  'Pan-Cancer' = "All",
  'Pan-Cancer ## Alkaloid ## Platinum' = "Chemotherapy",
  'Pan-Cancer ## Alkylating ## Anthracycline' = "Chemotherapy",
  'Pan-Cancer ## Anti-AR' = "Hormonal",
  'Pan-Cancer ## Anti-CTLA-4 ## Anti-PD-1' = 'Immunotherapy',
  'Pan-Cancer ## Anti-EGFR' = "Targeted",
  'Pan-Cancer ## Anti-PD-1' = 'Immunotherapy',
  'Pan-Cancer ## Anti-VEGF ## Platinum ## Pyrimidine (ant)agonist' = "Chemotherapy",
  'Pan-Cancer ## Aromatase inhibitor' = "Hormonal" ,
  'Pan-Cancer ## Chemotherapy'= "Chemotherapy",
  'Pan-Cancer ## Chemotherapy ## Detoxificant' = "Chemotherapy",
  'Pan-Cancer ## Chemotherapy ## Targeted therapy' = "Targeted",
  'Pan-Cancer ## Folinic acid ## Platinum ## Pyrimidine (ant)agonist ## Topoisomerase inhibitor' = "Chemotherapy",
  'Pan-Cancer ## Hormonal therapy' = "Hormonal",
  'Pan-Cancer ## Hormonal therapy ## Targeted therapy' = "Targeted",
  'Pan-Cancer ## Immunotherapy' = 'Immunotherapy',
  'Pan-Cancer ## Multikinase inhibitor' = "Targeted",
  'Pan-Cancer ## Platinum ## Pyrimidine (ant)agonist' = "Chemotherapy",
  'Pan-Cancer ## Platinum ## Taxane' = "Chemotherapy",
  'Pan-Cancer ## Pyrimidine (ant)agonist' = "Chemotherapy",
  'Pan-Cancer ## Targeted therapy' = "Targeted",
  'Pan-Cancer ## Taxane' = "Chemotherapy",
  'Pancreas PAAD ## Chemotherapy ## Detoxificant' = "Chemotherapy",
  'Pancreas PAAD ## Folinic acid ## Platinum ## Pyrimidine (ant)agonist ## Topoisomerase inhibitor' = "Chemotherapy",
  'Prostate ## Anti-AR' = "Hormonal",
  'Prostate ## Chemotherapy' = "Chemotherapy",
  'Prostate ## Hormonal therapy'  = "Hormonal",
  'Prostate ## Taxane' = "Chemotherapy",
  'Skin Melanoma ## Anti-PD-1'= 'Immunotherapy',
  'Skin Melanoma ## Immunotherapy'= 'Immunotherapy',
  'Unknown primary (e.g. CUP) ## Chemotherapy' = "Chemotherapy")

In [ ]:
results_univariate <- 
fread(paste0(SHARE_DIR, "1_run_fishers_exact.csv")) %>% 
 unique() %>% 
 rw() %>% mu(therapy = type_map[[cohortGo]]) %>% ug() %>% 
 fi(!cohortGo %in% c("Pan-Cancer ## Anti-AR", "Prostate ## Hormonal therapy")) %>% 
 fi( ci_high - ci_low < 4)

In [ ]:
results_univariate %>% 
 rw() %>% mu(therapy = type_map[[cohortGo]]) %>% ug() %>% 
 gb(feature) %>% 
 su(non_response = sum(direction == "Non-Response")) %>% 
 ar(desc(non_response)) %>% 
 fi(grepl("driver", feature)) %>% head(20)
 #fi(grepl("geneset", feature), grepl("driver", feature)) %>% head(20)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)

In [ ]:
a <- 
results_univariate %>% 
 fi(feature == "hotspot_KRAS_position_25398284") %>% 
 ggplot(aes(x = or, y = fct_reorder(cohortGo, desc(or)), color = therapy)) + 
  geom_point(size = 3) +
  geom_errorbarh(aes(xmin = ci_low, xmax = ci_high), height = 0.2) +
  geom_vline(xintercept = 1, linetype = "dashed", color = "red") +
  labs( x = "Odds Ratio or Response", y = "",title = "KRAS Hotspot position 25398284") +
  theme_minimal()

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)

In [ ]:
a
ggsave( paste0(FIG_DIR, "example_KRAS_hotspot.png"), width = 10, height = 5)

In [ ]:
b <- 
results_univariate %>% 
 fi(feature == "driver_KRAS") %>% 
 ggplot(aes(x = or, y = fct_reorder(cohortGo, desc(or)), color = therapy)) + 
  geom_point(size = 3) +
  geom_errorbarh(aes(xmin = ci_low, xmax = ci_high), height = 0.2) +
  geom_vline(xintercept = 1, linetype = "dashed", color = "red") +
  labs(x = "Odds ratio of Response",title = "KRAS Driver") +
  theme_minimal() 

In [ ]:
b
ggsave( paste0(FIG_DIR, "example_KRAS_driver.png"), width = 10, height = 5)

In [ ]:
c <- 
results_univariate %>% 
 fi(feature == "driver_pathways_gt4",  (!grepl("Pan-Cancer", cohortGo) | cohortGo == "Pan-Cancer")) %>%
 #fi(feature == "driver_pathways_gt4") %>%
 ggplot(aes(x = or, y = fct_reorder(cohortGo, desc(or)), color = therapy)) + 
  geom_point(size = 3) +
  geom_errorbarh(aes(xmin = ci_low, xmax = ci_high), height = 0.2) +
  geom_vline(xintercept = 1, linetype = "dashed", color = "red") +
  labs( x = "Odds ratio of Response", title = "Drivers in more that 4 Cancer pathways") +
  theme_minimal()

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
c
ggsave( paste0(FIG_DIR, "example_many_pathways_driver.png"), width = 10, height = 5)

In [ ]:
d <- 
results_univariate %>% 
 fi(feature == "rna_geneset_KEGG_ABC_TRANSPORTERS_gt75", !grepl("Pan-Cancer", cohortGo)) %>% 
 ggplot(aes(x = or, y = fct_reorder(cohortGo, desc(or)), color = therapy)) + 
  geom_point(size = 3) +
  geom_errorbarh(aes(xmin = ci_low, xmax = ci_high), height = 0.2) +
  geom_vline(xintercept = 1, linetype = "dashed", color = "red") +
  labs(
    x = "Odds ratio of Response",y = "Study",title = "High Multi-Drug Resistance Expression"
  ) +
  theme_minimal()

In [ ]:
d
ggsave( paste0(FIG_DIR, "example_high_drug_resistance.png"), width = 10, height = 5)

In [ ]:
e <- 
results_univariate %>% 
 fi( grepl("driver_ABC", feature)) %>% 
 ggplot(aes(x = or, y = fct_reorder(cohortGo, desc(or)), color = therapy)) + 
  geom_point(size = 3) +
  geom_errorbarh(aes(xmin = ci_low, xmax = ci_high), height = 0.2) +
  geom_vline(xintercept = 1, linetype = "dashed", color = "red") +
  labs(
    x = "Odds ratio of Response",title = "Driver ABCB1"
  ) +
  theme_minimal()

In [ ]:
e
ggsave( paste0(FIG_DIR, "example_driver_abcb1.png"), width = 10, height = 3)

In [ ]:
f <- 
results_univariate %>% 
 fi(ci_high - ci_low < 3) %>%
 fi( grepl("driver_TP53", feature)) %>% 
 ggplot(aes(x = or, y = fct_reorder(cohortGo, desc(or)), color = therapy)) + 
  geom_point(size = 3) +
  geom_errorbarh(aes(xmin = ci_low, xmax = ci_high), height = 0.2) +
  geom_vline(xintercept = 1, linetype = "dashed", color = "red") +
  labs(x = "Odds ratio of Response",title = "Driver TP53", y = "") +
  theme_minimal()

In [ ]:
f
ggsave( paste0(FIG_DIR, "example_driver_tp53.png"), width = 10, height = 5)